In [3]:
import os

In [4]:
%pwd

'/Users/sunnyg/Desktop/End-to-end-Kidney_AI/research'

In [5]:
os.chdir("../")

In [6]:
%pwd

'/Users/sunnyg/Desktop/End-to-end-Kidney_AI'

In [7]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
# frozen is true to prevent modification of the dataclass after creation
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list

In [8]:
from CNN_Classifier.constants import *
# import everything
from CNN_Classifier.utils.common import read_yaml, create_directories  
import tensorflow as tf 

In [9]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = Path("config/config.yaml"),
        params_filepath = Path("params.yaml")):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "kidney-ct-scan-image")
        create_directories([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )

        return training_config

In [10]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [12]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config

    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )

        # generate training and validation data

    def train_valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.20
        )

        # this means that 20% of the data will be used for validation and 80% for training

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )
        # this means that the images will be resized to the target size and the batch size will be set to the specified value

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        # this means that the validation data will be generated from the training data with a 20% split

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )

        # this if-else statement checks if augmentation is set to true or false. If true, it applies the specified augmentations to the training data. If false, it uses the same generator as the validation data.

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

    def train(self):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        # this means that the steps per epoch and validation steps will be calculated based on the number of samples and batch size

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )

In [19]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train()
    
except Exception as e:
    raise e

[2025-03-24 12:54:31,226: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-03-24 12:54:31,243: INFO: common: yaml file: params.yaml loaded successfully]
[2025-03-24 12:54:31,246: INFO: common: created directory at: artifacts]
[2025-03-24 12:54:31,250: INFO: common: created directory at: artifacts/training]
Found 93 images belonging to 2 classes.
Found 372 images belonging to 2 classes.
Epoch 1/8
23/23 [==============================] - 47s 2s/step - loss: 11.1972 - accuracy: 0.5899 - val_loss: 7.9889 - val_accuracy: 0.4375
Epoch 2/8
23/23 [==============================] - 52s 2s/step - loss: 5.3497 - accuracy: 0.7191 - val_loss: 6.6966 - val_accuracy: 0.4375
Epoch 3/8
23/23 [==============================] - 163s 7s/step - loss: 5.6415 - accuracy: 0.6798 - val_loss: 9.7336 - val_accuracy: 0.4750
Epoch 4/8
23/23 [==============================] - 54s 2s/step - loss: 3.3711 - accuracy: 0.7669 - val_loss: 11.1883 - val_accuracy: 0.4750
Epoch 5/8
23/23 [=============